In [ ]:
import sys
from collections import deque

jumpEnergy = deque()

N = int(sys.stdin.readline().strip())

for i in range(N-1):
    smallJump, bigJump = map(int, sys.stdin.readline().strip().split())
    jumpEnergy.append({'smallJump': smallJump, 'bigJump': bigJump})

K = int(sys.stdin.readline().strip())
superJump = False
checkStone = 0
useEnergy = 0

def chooseJump(i, checkStone, N, jumpEnergy, K):
    global superJump
    if N-2 == checkStone:
        return 1, jumpEnergy[i]['smallJump']

    # 범위 검사
    next1_small = jumpEnergy[i+1]['smallJump']
    next2_small = jumpEnergy[i+2]['smallJump']

    current_small = jumpEnergy[i]['smallJump']
    current_big = jumpEnergy[i]['bigJump']

    if superJump == True or current_small + next1_small + next2_small < K or current_small + next1_small < K:
        if current_small + next1_small < current_big:
            return 1, current_small
        else:
            return 2, current_big
    else:
        superJump = True
        return 3, K

i = 0
while N-1 > checkStone:
    jump, energy = chooseJump(i, checkStone, N, jumpEnergy, K)
    checkStone += jump
    useEnergy += energy
    i += 1

print(useEnergy)


# 점화식
1. 최종 결과부터 확인
2. 이전 값이 1개면 1계 점화식 2개면 2계 점화식이라고 한다.
3. 입력 예제를 바탕으로 규칙성을 찾아내기
4. 기저 값을 선횅 시키기

## 구현방식

- 중복된 연산을 저장하는 과정을 구현방법에 따라, 메모이제이션 (Memoization) 혹은 테뷸레이션 (Tabulation) 이라고 하고 , 저장된 메모리를 캐시(Cache) 라고한다.

### 1. Top-Down - 재귀
- top-down에서는 캐싱하는 방식을 메모이제이션 (Memoization) 이라고 하는데, 이미 계산된 값이 캐시에 있으면 꺼내는 방식

### 2. Bottom-Top - 반복
- bottom-up 에서는 캐싱하는 방식을 테뷸레이션 (Tabulation) 이라고 하는데, 작은 값부터 캐시에 값을 채워나가는 방식


In [ ]:
import sys
input = sys.stdin.read

data = input().split()

N = int(data[0])
index = 1
jumpEnergy = []
for i in range(N-1):
    smallJump, bigJump = map(int, data[index:index+2])
    jumpEnergy.append((smallJump, bigJump))
    index += 2

K = int(data[index])

# DP 테이블 초기화
dp_no_super = [float('inf')] * N  # 슈퍼 점프를 사용하지 않았을 때
dp_super_used = [float('inf')] * N  # 슈퍼 점프를 사용했을 때

dp_no_super[0] = 0  # 시작 지점

for i in range(N-1):
    # 슈퍼 점프를 사용하지 않은 상태 업데이트
    if i + 1 < N:
        dp_no_super[i+1] = min(dp_no_super[i+1], dp_no_super[i] + jumpEnergy[i][0])
    if i + 2 < N:
        dp_no_super[i+2] = min(dp_no_super[i+2], dp_no_super[i] + jumpEnergy[i][1])
    
    # 슈퍼 점프를 사용한 상태 업데이트
    # 슈퍼 점프를 이번에 처음 사용하는 경우
    if i + 3 < N:
        dp_super_used[i+3] = min(dp_super_used[i+3], dp_no_super[i] + K)
    
    # 이미 슈퍼 점프를 사용한 상태에서의 계속 업데이트
    if i + 1 < N:
        dp_super_used[i+1] = min(dp_super_used[i+1], dp_super_used[i] + jumpEnergy[i][0])
    if i + 2 < N:
        dp_super_used[i+2] = min(dp_super_used[i+2], dp_super_used[i] + jumpEnergy[i][1])

# 결과 출력: N-1번째 돌에 도착하는 최소 에너지
result = min(dp_no_super[N-1], dp_super_used[N-1])
print(result)

# 정답 코드 분석
1. 최소값을 찾아주는 문제이기에 무한대 값으로 초기 설정
2. min으로 index 마다 가능한 범위를 지속적으로 비교를 하면서 더 적은 값으로 업데이트 되도록 로직을 수정함
3. index 값은 서있는 돌의 위치 값으로 생각하면 됨